In [31]:
import geopandas as gpd
from lakedata import georeader, waterquality
from shapely.geometry import Point
from numpy import isfinite

In [38]:
nhd_dir = "../datafiles/Shape/"
eco_dir = "../waeco/"
lakedata = georeader.getNHDWaterbodies(nhd_dir)
ecodata  = waterquality.loadWaterQualityData(eco_dir+'phytoplankton2017_11_6.csv', eco_dir+'EIMLocation.csv')
ecodata = ecodata[isfinite(ecodata.Latitude_Decimal_Degrees) | isfinite(ecodata.Longitude_Decimal_Degrees)]

In [13]:
lakedata.head()

,PERMANENT_,FDATE,RESOLUTION,GNIS_ID,GNIS_NAME,AREASQKM,ELEVATION,REACHCODE,FTYPE,FCODE,...,DESCRIPTIO,HYDROGRAPH,INUNDATION,KCODE,OPERATIONA,PIPELINETY,POSITIONAL,RELATIONSH,RESERVOIRT,STAGE
0,43938735,2016-03-24,2,None,None,0.004416,None,17060108009772,436,43624,...,Reservoir; Reservoir Type = Treatment,None,None,"436,RET417",None,None,None,None,Treatment,None
1,51681191,2002-08-14,2,None,None,0.009000,None,17010306105790,436,43624,...,Reservoir; Reservoir Type = Treatment,None,None,"436,RET417",None,None,None,None,Treatment,None
2,51681387,2013-12-26,2,None,None,0.005190,None,17010306105796,436,43624,...,Reservoir; Reservoir Type = Treatment,None,None,"436,RET417",None,None,None,None,Treatment,None
3,52133257,2002-08-20,2,None,None,0.003000,None,17010214129309,436,43624,...,Reservoir; Reservoir Type = Treatment,None,None,"436,RET417",None,None,None,None,Treatment,None
4,57379485,2002-10-09,2,None,None,0.002000,None,17010305089159,436,43624,...,Reservoir; Reservoir Type = Treatment,None,None,"436,RET417",None,None,None,None,Treatment,None


In [14]:
ecodata.head()

,County,WRIA Number,Site,Lab Sample Number,Collect Date,Parameter,Dominance,loc_idx,Latitude_Decimal_Degrees,Longitude_Decimal_Degrees
94,Benton,37.0,Yakima Delta,L68576-2,09/06/2017,Achnanthes sp.,Present,300,46.23886,-119.22881
95,Benton,37.0,Yakima Delta,L68576-2,09/06/2017,Actinastrum sp.,Present,300,46.23886,-119.22881
96,Benton,37.0,Yakima Delta,L68576-2,09/06/2017,Amphora sp.,Present,300,46.23886,-119.22881
97,Benton,37.0,Yakima Delta,L68576-2,09/06/2017,Anabaena: trichome irregularly twisted,Present,300,46.23886,-119.22881
98,Benton,37.0,Yakima Delta,L68576-2,09/06/2017,Ankistrodesmus sp.,Present,300,46.23886,-119.22881


In [40]:
eco_geom = [Point(xy) for xy in zip(ecodata.Longitude_Decimal_Degrees, ecodata.Latitude_Decimal_Degrees)]
ecodata = gpd.GeoDataFrame(data=ecodata, geometry=eco_geom, crs={'init': 'epsg:4326'})

In [41]:
ecodata_withlake = gpd.sjoin(ecodata, lakedata.to_crs(ecodata.crs), how='left')

In [49]:
groups = ecodata_withlake.groupby(by=['Site', 'Collect Date'])

AttributeError: Cannot access callable attribute 'groupby' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [88]:
ecodata.pivot(index=['Site', 'Collect Date'], columns='Parameter', values='Dominance')

Parameter,Acanthese sp.,Achnanthes sp.,Achnanthidium sp.,Actinastrum sp.,Additional Info,Algae,Amphora sp.,Anabaena sp.,Anabaena: trichome irregularly twisted,Anabaena: trichome other,...,Unicellular Green 4-7 microns,Unicellular Green 8-10 microns,Unidentified Blu-Grn,Unidentified Blue-Green,Unidentified Diatom,Urosolenia sp.,Volvox sp.,Woronichinia sp.,Xanthidium sp.,Zygnema sp.
index_left,,,,,,,,,,,,,,,,,,,,,
94,None,Present,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
95,None,None,None,Present,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
96,None,None,None,None,None,None,Present,None,None,None,...,None,None,None,None,None,None,None,None,None,None
97,None,None,None,None,None,None,None,None,Present,None,...,None,None,None,None,None,None,None,None,None,None
98,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
99,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
100,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
101,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
102,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [83]:
#groups = ecodata.groupby(['Site', 'Collect Date', 'Parameter'], group_keys=True)
#len(groups)